# Cleaning duplicates from catalog

We have a catalog that contains duplicated entries, where duplicates may present different flux values depending on the amount of overlaping observations.
We want to generate a unique sources catalog, with sources with the best signal-to-noise ratio -- the *primary sources* -- among the duplicates.

In [1]:
from xmatch import xmatch

In [2]:
help(xmatch)

Help on function xmatch in module xmatch.xmatchi:

xmatch(catalog_A, catalog_B, columns_A=None, columns_B=None, radius=None, separation_unit='arcsec', method='gc', parallel=False, nprocs=None, snr_column=None)
    Input:
     - catalog_A, catalog_B : ~pandas.DataFrame
             DFs containing (at least) the columns 'ra','dec','id'
     - columns_A, columns_B : dict mapping 'ra','dec','id' columns
            In case catalog(s) have different column names for 'ra','dec','id';
            e.g, {'ra':'RA', 'dec':'Dec', 'id':'ObjID'}
    
    Output:
     - matched_catalog : ~pandas.DataFrame



In [3]:
import pandas

df = pandas.read_csv('table_flux_detections_lamassa.csv', sep=';')
df.head()

,RA,DEC,NH,ENERGY_SLOPE,ENERGY_SLOPE_ERROR,nufnu_5keV(erg.s-1.cm-2),nufnu_error_5keV(erg.s-1.cm-2),nufnu_0.5keV(erg.s-1.cm-2),nufnu_error_0.5keV(erg.s-1.cm-2),upper_limit_0.5keV(erg.s-1.cm-2),nufnu_1.5keV(erg.s-1.cm-2),nufnu_error_1.5keV(erg.s-1.cm-2),upper_limit_1.5keV(erg.s-1.cm-2),nufnu_4.5keV(erg.s-1.cm-2),nufnu_error_4.5keV(erg.s-1.cm-2),upper_limit_4.5keV(erg.s-1.cm-2)
0,01:26:58.792,-01:14:17.856,3.670000e+20,-0.020,+0.40/-0.27,2.439260e-13,4.964830e-14,3.611210e-14,1.392740e-14,-999.0,4.836250e-14,2.150750e-14,-9.990000e+02,2.181130e-13,6.132380e-14,-9.990000e+02
1,01:26:42.217,-01:14:06.962,3.710000e+20,1.768,+0.46/-0.28,4.895280e-14,6.875400e-15,2.440420e-13,4.657340e-14,-999.0,1.617730e-13,3.816380e-14,-9.990000e+02,5.194850e-14,2.435380e-14,-9.990000e+02
2,01:26:32.964,-01:12:19.438,3.730000e+20,0.800,-999/-999,4.167940e-14,1.431790e-14,4.635930e-14,1.967250e-14,-999.0,3.168120e-14,1.901230e-14,-9.990000e+02,1.329030e-14,-4.905500e-11,5.941000e-11
3,01:26:27.323,-01:18:13.705,3.780000e+20,0.800,-999/-999,4.036140e-14,1.418400e-14,6.748230e-14,2.371500e-14,-999.0,3.065140e-14,-3.227770e-11,7.901590e-11,1.285420e-14,-4.851900e-11,1.187750e-10
4,01:27:22.793,-01:15:59.595,3.620000e+20,0.800,-999/-999,3.750180e-14,1.379660e-14,3.113790e-14,1.661030e-14,-999.0,4.285510e-14,2.286080e-14,-9.990000e+02,1.199850e-14,-4.742740e-11,8.907140e-11


## SNR column

Let's define the objects' overall SNR estimate to be the ratio between columns `nufnu_5keV(erg.s-1.cm-2)` and `nufnu_error_5keV(erg.s-1.cm-2)`, which are the flux and error associated to the full band emission.

In [4]:
snr = df['nufnu_5keV(erg.s-1.cm-2)']/df['nufnu_error_5keV(erg.s-1.cm-2)']
df['snr'] = snr

**xmatch** needs columns 'ra','dec','id' to be defined. As well as the search radius.

In [5]:
df.reset_index(inplace=True)
df.rename(columns={'index':'ID'}, inplace=True)

In [6]:
df.head()

,ID,RA,DEC,NH,ENERGY_SLOPE,ENERGY_SLOPE_ERROR,nufnu_5keV(erg.s-1.cm-2),nufnu_error_5keV(erg.s-1.cm-2),nufnu_0.5keV(erg.s-1.cm-2),nufnu_error_0.5keV(erg.s-1.cm-2),upper_limit_0.5keV(erg.s-1.cm-2),nufnu_1.5keV(erg.s-1.cm-2),nufnu_error_1.5keV(erg.s-1.cm-2),upper_limit_1.5keV(erg.s-1.cm-2),nufnu_4.5keV(erg.s-1.cm-2),nufnu_error_4.5keV(erg.s-1.cm-2),upper_limit_4.5keV(erg.s-1.cm-2),snr
0,0,01:26:58.792,-01:14:17.856,3.670000e+20,-0.020,+0.40/-0.27,2.439260e-13,4.964830e-14,3.611210e-14,1.392740e-14,-999.0,4.836250e-14,2.150750e-14,-9.990000e+02,2.181130e-13,6.132380e-14,-9.990000e+02,4.913079
1,1,01:26:42.217,-01:14:06.962,3.710000e+20,1.768,+0.46/-0.28,4.895280e-14,6.875400e-15,2.440420e-13,4.657340e-14,-999.0,1.617730e-13,3.816380e-14,-9.990000e+02,5.194850e-14,2.435380e-14,-9.990000e+02,7.119993
2,2,01:26:32.964,-01:12:19.438,3.730000e+20,0.800,-999/-999,4.167940e-14,1.431790e-14,4.635930e-14,1.967250e-14,-999.0,3.168120e-14,1.901230e-14,-9.990000e+02,1.329030e-14,-4.905500e-11,5.941000e-11,2.911000
3,3,01:26:27.323,-01:18:13.705,3.780000e+20,0.800,-999/-999,4.036140e-14,1.418400e-14,6.748230e-14,2.371500e-14,-999.0,3.065140e-14,-3.227770e-11,7.901590e-11,1.285420e-14,-4.851900e-11,1.187750e-10,2.845558
4,4,01:27:22.793,-01:15:59.595,3.620000e+20,0.800,-999/-999,3.750180e-14,1.379660e-14,3.113790e-14,1.661030e-14,-999.0,4.285510e-14,2.286080e-14,-9.990000e+02,1.199850e-14,-4.742740e-11,8.907140e-11,2.718191


In [7]:
from astropy import units
cols = {'ra':'RA', 'dec':'DEC', 'id':'ID'}
radius = 6*units.arcsec

In [11]:
xcat = xmatch(df, df, columns_A=cols, columns_B=cols, radius=radius, snr_column='snr')

/home/chbrandt/.conda/envs/xmatch/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/chbrandt/.conda/envs/xmatch/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [12]:
xcat

A                                  B                       \
                RA            DEC    ID            RA            DEC    ID   
0     01:26:58.792  -01:14:17.856     0  01:26:58.795  -01:14:17.195    10   
1     01:26:42.217  -01:14:06.962     1  01:26:42.199  -01:14:07.136    11   
2     01:26:32.964  -01:12:19.438     2  01:26:32.618  -01:12:21.290    12   
3     01:26:27.323  -01:18:13.705     3  01:26:27.323  -01:18:13.705     3   
4     01:27:22.793  -01:15:59.595     4  01:27:22.695  -01:15:59.333    13   
5     01:25:44.247  -01:22:43.589     5  01:25:44.192  -01:22:44.154    15   
6     01:26:00.398  -01:20:38.631     6  01:26:00.389  -01:20:38.317    14   
7     01:25:35.964  -01:25:45.307     7  01:25:35.983  -01:25:41.160    16   
8     01:26:05.435  -01:26:52.958     8  01:26:05.435  -01:26:52.958     8   
9     01:26:11.419  -01:11:55.131     9  01:26:11.419  -01:11:55.131     9   
17    01:33:52.478  +01:13:42.379    17  01:33:52.466  +01:13:43.093    18   
19    01:33:48.007  +01:06:38.237    19  01:33:48.007  +01:06:38.237    19   
22    01:26:59.141  +01:12:48.229    22  01:26:59.117  +01:12:47.423    31   
23    01:27:13.837  +01:13:19.386    23  01:27:13.860  +01:13:19.311    32   
24    01:27:12.912  +01:14:19.917    24  01:27:13.375  +01:14:23.941    30   
25    01:26:50.315  +01:16:11.448    25  01:26:50.378  +01:16:11.311    29   
26    01:27:29.455  +01:08:31.981    26  01:27:29.470  +01:08:32.852    35   
27    01:26:38.993  +01:06:23.964    27  01:26:38.909  +01:06:25.247    36   
28    01:27:28.045  +01:21:38.720    28  01:27:28.140  +01:21:39.220    34   
33    01:27:11.462  +01:20:23.510    33  01:27:11.462  +01:20:23.510    33   
45    02:32:54.943  +00:26:19.598    45  02:32:54.943  +00:26:19.598    45   
46    02:33:01.240  +00:25:18.043    46  02:33:01.221  +00:25:18.531    65   
47    02:32:29.210  +00:25:48.913    47  02:32:29.139  +00:25:48.291    66   
48    02:32:28.323  +00:27:37.475    48  02:32:28.323  +00:27:37.475    48   
49    02:32:37.556  +00:32:11.219    49  02:32:37.556  +00:32:11.219    49   
50    02:32:44.432  +00:33:50.344    50  02:32:44.463  +00:33:48.929    70   
51    02:32:14.792  +00:28:59.919    51  02:32:14.792  +00:28:59.919    51   
52    02:33:14.764  +00:34:03.069    52  02:33:14.764  +00:34:03.069    52   
53    02:33:25.355  +00:29:17.613    53  02:33:25.375  +00:29:17.116    73   
54    02:33:18.535  +00:21:25.705    54  02:33:18.557  +00:21:25.565    74   
...            ...            ...   ...           ...            ...   ...   
1974  00:03:19.487  -00:17:14.356  1974  00:03:19.487  -00:17:14.356  1974   
1975  00:01:55.182  -00:21:31.759  1975  00:01:55.182  -00:21:31.759  1975   
1976  00:02:43.983  -00:33:13.514  1976  00:02:44.046  -00:33:16.814  1984   
1985  00:07:35.382  +00:47:44.178  1985  00:07:35.382  +00:47:44.178  1985   
1986  00:07:17.552  +00:46:41.214  1986  00:07:17.552  +00:46:41.214  1986   
1987  00:07:46.032  +00:52:09.220  1987  00:07:46.032  +00:52:09.220  1987   
1988  00:07:27.339  +00:40:02.392  1988  00:07:27.339  +00:40:02.392  1988   
1989  00:07:26.661  +00:55:10.824  1989  00:07:26.661  +00:55:10.824  1989   
1990  00:08:15.535  +00:45:13.651  1990  00:08:15.535  +00:45:13.651  1990   
1991  00:08:13.434  +00:43:09.575  1991  00:08:13.434  +00:43:09.575  1991   
1992  00:07:11.972  +00:40:02.155  1992  00:07:11.972  +00:40:02.155  1992   
1993  00:07:09.859  +00:53:29.098  1993  00:07:09.859  +00:53:29.098  1993   
1994  00:07:50.329  +00:58:12.877  1994  00:07:50.329  +00:58:12.877  1994   
2005  03:22:27.223  +00:54:05.197  2005  03:22:27.223  +00:54:05.197  2005   
2006  03:22:13.988  +00:55:15.498  2006  03:22:13.988  +00:55:15.498  2006   
2007  03:23:10.947  +00:54:16.267  2007  03:23:10.947  +00:54:16.267  2007   
2008  00:22:40.187  +01:18:16.671  2008  00:22:40.187  +01:18:16.671  2008   
2009  00:22:37.076  +01:15:35.687  2009  00:22:37.076  +01:15:35.687  2009   
2010  00:22:15.894  +01:17:14.968  2010

In [25]:
pcat = df.set_index('ID').loc[xcat[('B','ID')]]

In [26]:
pcat.head()

,RA,DEC,NH,ENERGY_SLOPE,ENERGY_SLOPE_ERROR,nufnu_5keV(erg.s-1.cm-2),nufnu_error_5keV(erg.s-1.cm-2),nufnu_0.5keV(erg.s-1.cm-2),nufnu_error_0.5keV(erg.s-1.cm-2),upper_limit_0.5keV(erg.s-1.cm-2),nufnu_1.5keV(erg.s-1.cm-2),nufnu_error_1.5keV(erg.s-1.cm-2),upper_limit_1.5keV(erg.s-1.cm-2),nufnu_4.5keV(erg.s-1.cm-2),nufnu_error_4.5keV(erg.s-1.cm-2),upper_limit_4.5keV(erg.s-1.cm-2),snr
ID,,,,,,,,,,,,,,,,,
10,01:26:58.795,-01:14:17.195,3.670000e+20,-0.020,+0.17/-0.14,5.346740e-13,5.346740e-14,7.917950e-14,1.500040e-14,-999.0,1.060040e-13,2.316250e-14,-9.990000e+02,4.780300e-13,6.606760e-14,-9.990000e+02,10.000000
11,01:26:42.199,-01:14:07.136,3.710000e+20,1.820,+0.18/-0.14,9.830070e-14,6.259200e-15,5.693350e-13,4.693000e-14,-999.0,3.317110e-13,3.624540e-14,-9.990000e+02,1.050400e-13,2.276120e-14,-9.990000e+02,15.704994
12,01:26:32.618,-01:12:21.290,3.730000e+20,0.800,-999/-999,4.567580e-14,1.022710e-14,3.811650e-14,1.211480e-14,-999.0,5.209630e-14,1.655820e-14,-9.990000e+02,1.361950e-14,-3.503620e-11,5.930200e-11,4.466154
3,01:26:27.323,-01:18:13.705,3.780000e+20,0.800,-999/-999,4.036140e-14,1.418400e-14,6.748230e-14,2.371500e-14,-999.0,3.065140e-14,-3.227770e-11,7.901590e-11,1.285420e-14,-4.851900e-11,1.187750e-10,2.845558
13,01:27:22.695,-01:15:59.333,3.620000e+20,1.131,+0.35/-0.24,5.606770e-14,8.933920e-15,6.344500e-14,1.552790e-14,-999.0,7.948320e-14,1.945320e-14,-9.990000e+02,5.599000e-14,1.943960e-14,-9.990000e+02,6.275823


In [27]:
pcat.describe()

,NH,ENERGY_SLOPE,nufnu_5keV(erg.s-1.cm-2),nufnu_error_5keV(erg.s-1.cm-2),nufnu_0.5keV(erg.s-1.cm-2),nufnu_error_0.5keV(erg.s-1.cm-2),upper_limit_0.5keV(erg.s-1.cm-2),nufnu_1.5keV(erg.s-1.cm-2),nufnu_error_1.5keV(erg.s-1.cm-2),upper_limit_1.5keV(erg.s-1.cm-2),nufnu_4.5keV(erg.s-1.cm-2),nufnu_error_4.5keV(erg.s-1.cm-2),upper_limit_4.5keV(erg.s-1.cm-2),snr
count,1.079000e+03,1079.000000,1.079000e+03,1.079000e+03,1.079000e+03,1.079000e+03,1.079000e+03,1.079000e+03,1.079000e+03,1.079000e+03,1.079000e+03,1.079000e+03,1.079000e+03,1079.000000
mean,4.597396e+20,0.745594,1.123217e-11,2.654995e-14,6.604700e-12,-9.984959e-13,-9.443744e+02,8.864537e-12,-3.594645e-12,-9.323383e+02,1.078167e-11,-5.504448e-12,-9.064143e+02,inf
std,3.092824e+20,0.356392,1.674126e-10,1.769790e-13,1.138800e-10,6.269327e-12,2.272330e+02,1.411701e-10,4.566946e-11,2.494172e+02,1.610108e-10,2.576469e-11,2.898256e+02,NaN
min,1.920000e+20,-1.016000,2.158200e-15,0.000000e+00,0.000000e+00,-9.590400e-11,-9.990000e+02,0.000000e+00,-1.420850e-09,-9.990000e+02,0.000000e+00,-3.388560e-10,-9.990000e+02,2.085714
25%,2.635000e+20,0.799000,1.526155e-14,3.328200e-15,6.187325e-15,2.450835e-15,-9.990000e+02,1.020140e-14,3.244630e-15,-9.990000e+02,1.314650e-14,4.021325e-15,-9.990000e+02,3.206421
50%,3.190000e+20,0.800000,2.819340e-14,7.111100e-15,1.502440e-14,5.962520e-15,-9.990000e+02,2.216380e-14,8.239640e-15,-9.990000e+02,2.913140e-14,1.090750e-14,-9.990000e+02,4.241186
75%,5.030000e+20,0.800000,6.462540e-14,1.280000e-14,3.922200e-14,1.156610e-14,-9.990000e+02,5.588570e-14,1.602425e-14,-9.990000e+02,6.910105e-14,2.107255e-14,-9.990000e+02,6.547396
max,1.180000e+21,3.321000,3.779140e-09,2.989680e-12,3.317730e-09,2.506400e-12,1.657050e-10,3.602330e-09,3.183580e-12,2.372670e-10,3.706760e-09,4.577300e-12,3.571620e-10,inf


In [28]:
len(pcat)

1079

In [29]:
len(df)

2036

In [30]:
pcat.reset_index(inplace=True)
del pcat['snr']
pcat.head()

,ID,RA,DEC,NH,ENERGY_SLOPE,ENERGY_SLOPE_ERROR,nufnu_5keV(erg.s-1.cm-2),nufnu_error_5keV(erg.s-1.cm-2),nufnu_0.5keV(erg.s-1.cm-2),nufnu_error_0.5keV(erg.s-1.cm-2),upper_limit_0.5keV(erg.s-1.cm-2),nufnu_1.5keV(erg.s-1.cm-2),nufnu_error_1.5keV(erg.s-1.cm-2),upper_limit_1.5keV(erg.s-1.cm-2),nufnu_4.5keV(erg.s-1.cm-2),nufnu_error_4.5keV(erg.s-1.cm-2),upper_limit_4.5keV(erg.s-1.cm-2)
0,10,01:26:58.795,-01:14:17.195,3.670000e+20,-0.020,+0.17/-0.14,5.346740e-13,5.346740e-14,7.917950e-14,1.500040e-14,-999.0,1.060040e-13,2.316250e-14,-9.990000e+02,4.780300e-13,6.606760e-14,-9.990000e+02
1,11,01:26:42.199,-01:14:07.136,3.710000e+20,1.820,+0.18/-0.14,9.830070e-14,6.259200e-15,5.693350e-13,4.693000e-14,-999.0,3.317110e-13,3.624540e-14,-9.990000e+02,1.050400e-13,2.276120e-14,-9.990000e+02
2,12,01:26:32.618,-01:12:21.290,3.730000e+20,0.800,-999/-999,4.567580e-14,1.022710e-14,3.811650e-14,1.211480e-14,-999.0,5.209630e-14,1.655820e-14,-9.990000e+02,1.361950e-14,-3.503620e-11,5.930200e-11
3,3,01:26:27.323,-01:18:13.705,3.780000e+20,0.800,-999/-999,4.036140e-14,1.418400e-14,6.748230e-14,2.371500e-14,-999.0,3.065140e-14,-3.227770e-11,7.901590e-11,1.285420e-14,-4.851900e-11,1.187750e-10
4,13,01:27:22.695,-01:15:59.333,3.620000e+20,1.131,+0.35/-0.24,5.606770e-14,8.933920e-15,6.344500e-14,1.552790e-14,-999.0,7.948320e-14,1.945320e-14,-9.990000e+02,5.599000e-14,1.943960e-14,-9.990000e+02


In [31]:
pcat.to_csv('table_flux_detections_lamassa_unique.csv', index=False)